In [16]:
#loading libraries 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
# from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import VotingClassifier
from sklearn import model_selection
from sklearn.metrics import confusion_matrix
from sklearn. preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
#readinf the dataset
df_train=pd.read_csv("Train_Clean.csv")
df_test=pd.read_csv("Test_Clean.csv")


In [3]:
#splitting the dataset into train and test with ratio of 80:20
x_train,x_test,y_train,y_test=train_test_split(df_train["Sentences"],df_train[["anger","disgust","fear","sadness","surprise","happiness","neutral"]],test_size=0.2,random_state=42)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((6240,), (1560,), (6240, 7), (1560, 7))

In [5]:
#vectorizing the urdu statements using the Term frequency and inverse document frequency
from sklearn.feature_extraction.text import TfidfVectorizer
max_feature_num = 50000
vectorizer = TfidfVectorizer(max_features=max_feature_num)
train_vecs = vectorizer.fit_transform(x_train)
test_vecs = TfidfVectorizer(max_features=max_feature_num, vocabulary=vectorizer.vocabulary_).fit_transform(x_test)
# visualizing the shape of train and test vecs
train_vecs.shape,test_vecs.shape


((6240, 18254), (1560, 18254))

In [7]:
from skmultilearn.problem_transform import LabelPowerset


In [15]:
#defining top models that showed the best performance keeping the F-1 Score metruc as the base metric 
"""
the models are 
1. Gaussian NB
2.KNN
3.SVM
4.DTC
5.Logistic Regression 

"""
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
def build_model(model,mlb_estimator,xtrain,ytrain,xtest,ytest):
    clf = mlb_estimator(model)
    clf.fit(xtrain,ytrain)
    clf_predictions = clf.predict(xtest)
    acc = accuracy_score(ytest,clf_predictions)
    f_1_score = f1_score(ytest,clf_predictions,average="weighted")
    result = {"accuracy:":acc,"f1_score":f_1_score}
    return model,result

In [20]:
#building the model based on Gaussian Nave Bayes
model1,result1=build_model(GaussianNB(),LabelPowerset,train_vecs,y_train,test_vecs,y_test)

model2,result2=build_model(LinearSVC(),LabelPowerset,train_vecs,y_train,test_vecs,y_test)

# model3=build_model(LogisticRegression(),LabelPowerset,train_vecs,y_train,test_vecs,y_test)

model3,result3=build_model(DecisionTreeClassifier(),LabelPowerset,train_vecs,y_train,test_vecs,y_test)

model4,result4=build_model(KNeighborsClassifier(),LabelPowerset,train_vecs,y_train,test_vecs,y_test)


In [39]:
estimators=[]
#adding weak 6 gnb
estimators.append(("GNB1",GaussianNB()))
estimators.append(("GNB2",GaussianNB()))
estimators.append(("GNB3",GaussianNB()))
estimators.append(("GNB4",GaussianNB()))
estimators.append(("GNB5",GaussianNB()))
estimators.append(("GNB6",GaussianNB()))
#adding 6 weak svc 
estimators.append(("LSVC1",LinearSVC()))
estimators.append(("LSVC2",LinearSVC()))
estimators.append(("LSVC3",LinearSVC()))
estimators.append(("LSVC4",LinearSVC()))
estimators.append(("LSVC5",LinearSVC()))
estimators.append(("LSVC6",LinearSVC()))
#adding 6 weak dtc
estimators.append(("DTC1",DecisionTreeClassifier()))
estimators.append(("DTC2",DecisionTreeClassifier()))
estimators.append(("DTC3",DecisionTreeClassifier()))
estimators.append(("DTC4",DecisionTreeClassifier()))
estimators.append(("DTC5",DecisionTreeClassifier()))
estimators.append(("DTC6",DecisionTreeClassifier()))
#adding 6 weak knn
estimators.append(("KNN1",KNeighborsClassifier()))
estimators.append(("KNN2",KNeighborsClassifier()))
estimators.append(("KNN3",KNeighborsClassifier()))
estimators.append(("KNN4",KNeighborsClassifier()))
estimators.append(("KNN5",KNeighborsClassifier()))
estimators.append(("KNN6",KNeighborsClassifier()))

In [40]:
#defining the ensemble model
ensemble_model,result=build_model(VotingClassifier(estimators),LabelPowerset,train_vecs,y_train,test_vecs,y_test)

In [41]:
ensemble_model


VotingClassifier(estimators=[('GNB1', GaussianNB()), ('GNB2', GaussianNB()),
                             ('GNB3', GaussianNB()), ('GNB4', GaussianNB()),
                             ('GNB5', GaussianNB()), ('GNB6', GaussianNB()),
                             ('LSVC1', LinearSVC()), ('LSVC2', LinearSVC()),
                             ('LSVC3', LinearSVC()), ('LSVC4', LinearSVC()),
                             ('LSVC5', LinearSVC()), ('LSVC6', LinearSVC()),
                             ('DTC1', DecisionTreeClassifier()),
                             ('DTC2', DecisionTreeClassifier()),
                             ('DTC3', DecisionTreeClassifier()),
                             ('DTC4', DecisionTreeClassifier()),
                             ('DTC5', DecisionTreeClassifier()),
                             ('DTC6', DecisionTreeClassifier()),
                             ('KNN1', KNeighborsClassifier()),
                             ('KNN2', KNeighborsClassifier()),
                             ('KNN3', KNeighborsClassifier()),
                             ('KNN4', KNeighborsClassifier()),
                             ('KNN5', KNeighborsClassifier()),
                             ('KNN6', KNeighborsClassifier())])

In [42]:
result

{'accuracy:': 0.5948717948717949, 'f1_score': 0.6705577384664321}